In [10]:
import datetime, warnings, scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#import geopandas as gdp
import seaborn as sns
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
# from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

In [113]:
df_airlines = pd.read_csv('Data/airlines.csv')
df_airports = pd.read_csv('Data/airports.csv')
df_flights = pd.read_csv('Data/flights_train.csv')
df_flights_test = pd.read_csv('Data/flights_test.csv')

In [114]:
def change_time(gg):
    time = "0000"[len(str(gg))-1:-1] + str(gg)
    return time

In [155]:
test = df_flights.loc[:100, 'SCHEDULED_DEPARTURE':'DEPARTURE_TIME']

delays = []

for i in range (test.shape[0]):
    if(abs(test['DEPARTURE_TIME'][i] - test['SCHEDULED_DEPARTURE'][i]) < 2000):
        delays.append(test['DEPARTURE_TIME'][i] - test['SCHEDULED_DEPARTURE'][i])
    elif (test['DEPARTURE_TIME'][i] > test['SCHEDULED_DEPARTURE'][i]):
        delays.append((test['DEPARTURE_TIME'][i] - 2360) - test['SCHEDULED_DEPARTURE'][i])
    else:
        delays.append((2360 - test['SCHEDULED_DEPARTURE'][i]) + test['DEPARTURE_TIME'][i])

        
for i in range (len(delays)):
    if delays[i] > 99:     #212 becomes 132 minutes, 112 becomes 72 minutes
        hour = int(delays[i] / 100)
        minutes = delays[i] % 100
        output = hour*60 + minutes
        delays[i] = output
    elif delays[i] > 59 and delays[i] <= 99:  #96 becomes 56, 83 becomes 43
        output = 60 - (100 - delays[i])
        delays[i] = output

#now for negative values
for i in range (len(delays)):
    if delays[i] < -99:     #-410 becomes -250 minutes
        hour = int(delays[i] / 100)
        minutes = -delays[i] % 100
        output = hour*60 - minutes
        delays[i] = output
    elif delays[i] < -59 and delays[i] >= -99:  #96 becomes 56, 83 becomes 43
        output = delays[i] + 40
        delays[i] = output
        
test["DEPARTURE_DELAY"] = delays
test['SCHEDULED_DEPARTURE'] = test['SCHEDULED_DEPARTURE'].apply(lambda row: change_time(row))
test['DEPARTURE_TIME'] = test['DEPARTURE_TIME'].apply(lambda row: change_time(int(row)))
test

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY
0,0005,2354,-11
1,0010,0222,132
2,0020,0018,-2
3,0020,0015,-5
4,0025,0024,-1
...,...,...,...
96,0535,0530,-5
97,0538,0634,56
98,0538,0535,-3
99,0540,0536,-4
